In [13]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from ir_measures import *

In [14]:
ensure_pyterrier_is_loaded()
tira = Client()


In [15]:
pt_dataset = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-01-20240426-training")

In [24]:
pt.Experiment(
    [
        tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training'),
        tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-01-20240423-training'),
        tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training'),
        tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-01-20240423-training'),
        pt.transformer.get_transformer(pt.io.read_results('sparse-cross-encoder-max-score.lag1')),
    ],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
        # "3-max-avg",
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
).sort_values(by="nDCG@10", ascending=False)

,name,nDCG(judged_only=True)@10,nDCG(judged_only=True),nDCG@10,nDCG
4,WSum,0.454667,0.480725,0.251512,0.355077
3,RankZephyr,0.455142,0.482076,0.246549,0.352920
1,Sparse Cross-Encoder,0.448109,0.474784,0.220869,0.336719
2,ColBERT,0.450091,0.476047,0.216457,0.330096
0,BM25,0.444411,0.472638,0.199380,0.317285
